# RFM-анализ

In [14]:
import pandahouse as ph
import pandas as pd

Нужно написать функцию, которая будет отправлять текст SQL-запроса, в нашем случае - запроса к ClickHouse, на сервер, а по выполнении запроса забирать его результаты в каком-то виде.

In [9]:
connection = {'host': 'http://clickhouse.beslan.pro:8080/',
                      #'database':'test',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

RFM-анализ, в основе которого лежат поведенческие факторы групп или сегментов клиентов. Данный метод анализа позволяет изучить поведение пользователей и то, как они совершают платежи. В результате вы получите ценные инсайты по построению директ-маркетинга для вашей компании.​ Также RFM-сегментация поможет вам применять в коммуникациях особый подход к каждой группе клиентов.

## Что такое RFM-анализ и зачем он нужен маркетологу
RFM-анализ — метод анализа, позволяющий сегментировать клиентов по частоте и сумме покупок и выявлять тех, которые приносят больше денег.

Аббревиатура RFM расшифровывается:

- Recency — давность (как давно ваши клиенты что-то у вас покупали);
- Frequency — частота (как часто они у вас покупают);
- Monetary — деньги (общая сумма покупок).

По этим признакам можно разделить всех ваших клиентов на группы, понять, кто покупает у вас часто и много, кто — часто, но мало, а кто вообще давно ничего не покупал.

Как правило, небольшой процент пользователей реагирует на общие рекламные предложения. RFM-анализ и RFM-сегментация являются отличным методом прогнозирования реакции клиента и улучшения взаимодействия, а также повышение прибыли. RFM использует поведение покупателей, чтобы определить, как работать с каждой группой клиентов. 

Суть RFM-анализа в том, что мы разделяем всех клиентов на группы, в зависимости от того, как давно они сделали последнюю покупку, как часто покупали и насколько большой была сумма их заказов. По каждому из этих признаков мы выделяем по три равные группы. Затем присваиваем каждой группе числовое обозначение от 1 до 3.

**По давности заказа:**

- 1 — давние клиенты;
- 2 — относительно недавние клиенты;
- 3 — недавние клиенты.

**По частоте покупок:**

- 1 — покупает очень редко (единичные заказы);​
- 2 — покупает нечасто;
- 3 — покупает часто.

**По сумме покупок:**

- 1 — маленькая сумма покупок;
- 2 — средняя сумма покупок;
- 3 — большая сумма покупок.

Например, клиент «111» покупал давно, один раз и на маленькую сумму. Или клиент «333»: покупает часто, на большую сумму и последняя покупка была недавно. Это ваши лучшие клиенты.

In [120]:
query = """
  SELECT *
  FROM checks
  WHERE Rub > 0
  LIMIT 1000000
"""

In [121]:
df = ph.read_clickhouse(query=query, connection=connection_default)

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   Rub      1000000 non-null  uint32
 1   BuyDate  1000000 non-null  object
 2   UserID   1000000 non-null  uint64
dtypes: object(1), uint32(1), uint64(1)
memory usage: 19.1+ MB


In [123]:
df.BuyDate = pd.to_datetime(df.BuyDate)

In [124]:
df.head()

,Rub,BuyDate,UserID
0,3,2019-08-03,13957114790337189433
1,4,2019-08-03,13956859514850028320
2,1,2019-08-03,13956853330294515807
3,12,2019-08-03,13956766250445288278
4,15,2019-08-03,13956663941630845600


In [125]:
df.UserID.nunique()

718950

- определить последнюю дату покупки каждого клиента
- посчитать сколько раз заказывал каждый клиент
- общую сумму по каждому клиенту

In [161]:
rfm = df.assign(Count=1).groupby('UserID') \
              .agg({'BuyDate': max, 'Rub': sum, 'Count': sum}).reset_index()

С помощью формулы `TODAY()` - дата последней покупки мы рассчитываем какое количество дней назад были совершены последние покупки. Это и будут данные по нашей R(Recency) 

In [162]:
today = pd.Timestamp('2020') # возьмем начало 2020 для точки отсчета, а не pd.Timestamp.today()
rfm['BuyDate'] = (today - rfm['BuyDate']).dt.days

In [163]:
rfm.rename(columns={'BuyDate': 'R', 'Count': 'F', 'Rub': 'M'}, inplace=True)

In [129]:
rfm

,UserID,R,M,F
0,3116519148596,297,5,1
1,27344263464379,189,4,1
2,43996990388727,89,1,1
3,47290329858223,333,5,1
4,69805931616875,333,1,1
...,...,...,...,...
718945,18446621313467415614,152,20,1
718946,18446622348748244246,334,16,1
718947,18446628846419558718,90,50,4
718948,18446652928377370962,152,10,1


У нас есть все данные по Recency, Frequency (количество заказов) и Monetary (сумма). Сейчас мы будем присваивать значения от 1 до 3 (о чем мы уже говорили чуть выше) в соответствии с этими данными.

Начнем со значений для Recency. Первое, что нам нужно, это рассчитать как давно у нас покупали 33% и 66% клиентов.

In [130]:
rfm.R.quantile([.33, .66])

0.33    117.0
0.66    228.0
Name: R, dtype: float64

Теперь мы знаем, что 33% клиентов покупали наши товары менее 117 дней назад, а 66% — более 228 дней назад.

Соответственно, тем клиентам, кто покупал менее, чем 117 дней назад мы присваиваем наивысшее значение - 3. Тем, кто делал заказ от 151 до 228 дней назад - 2. И остальным, кто покупал более чем 228 дня назад мы присваиваем значение 1. 

Мы можем и нарезать квантили `pd.qcut(rfm.R, q=3, labels=range(3, 0, -1))`

In [164]:
rfm.R = rfm.R.apply(lambda x: 3 if x < 117 else 2 if 117 <= x <= 228 else 1)

По такому же принципу просчитываем значения для F(Frequency) и M(Mmonetary): сначала вычисляем сколько раз покупали 33% и 66% клиентов и по результатам проставляем значения от 1 до 3.

In [132]:
rfm.F.quantile([.33, .66]) # видим что большая часть пользователей совершила только одну покупку

0.33    1.0
0.66    1.0
Name: F, dtype: float64

In [165]:
rfm.F = rfm.F.apply(lambda x: 3 if x > 2 else 2 if  x == 2 else 1)

In [134]:
rfm.M.quantile([.33, .66])

0.33     4.0
0.66    15.0
Name: M, dtype: float64

In [166]:
rfm.M = rfm.M.apply(lambda x: 3 if x > 15 else 2 if 4 <= x <= 15 else 1)

И для большей наглядности нам осталось просчитать общее значение RFM, объединив все 3 значения по каждому клиенту в одну ячейку.

In [167]:
rfm = rfm.assign(RFM = rfm.R * 100 + rfm.F * 10 + rfm.M)

In [168]:
rfm.sort_values('RFM', ascending=False)

,UserID,R,M,F,RFM
359475,10704570005919383047,3,3,3,333
116830,3459934524673652148,3,3,3,333
116859,3460978299725389469,3,3,3,333
602236,15914960830994678801,3,3,3,333
697138,17948624464620080889,3,3,3,333
...,...,...,...,...,...
74393,2140535320606434197,1,1,1,111
508800,13861068737722211546,1,1,1,111
208801,6557669225612571818,1,1,1,111
508811,13861274725696009967,1,1,1,111


In [175]:
rfm.RFM.value_counts(normalize=True)

212    0.141580
211    0.120449
112    0.115000
111    0.096443
213    0.078445
113    0.062629
312    0.053089
223    0.046882
311    0.044536
222    0.039238
333    0.037694
323    0.030617
233    0.029583
313    0.029193
322    0.025519
332    0.010780
232    0.009688
123    0.008824
122    0.007448
221    0.006113
321    0.003867
121    0.001099
133    0.000686
132    0.000246
231    0.000174
331    0.000172
131    0.000007
Name: RFM, dtype: float64